## Linking and brushing with bokeh

Linking and brushing is a powerful method for exploratory data analysis.

One way to create linked plots in the notebook is to use Bokeh.

In [ ]:
import bokeh

In [ ]:
import numpy as np

In [ ]:
from astropy.table import Table

In [ ]:
sdss = Table.read('data/sdss_galaxies_qsos_50k.fits')

In [ ]:
sdss

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, gridplot, output_notebook, output_file, show


umg = sdss['u'] - sdss['g']
gmr = sdss['g'] - sdss['r']
rmi = sdss['r'] - sdss['i']
imz = sdss['i'] - sdss['z']

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(umg=umg, gmr=gmr, rmi=rmi,imz=imz))

We will output to a static html file. 

The output_notebook() function can output to the notebook, 
but with 50,000 points it really slows down.

In [ ]:
output_file('sdss_color_color.html')

In [ ]:
TOOLS = "pan,wheel_zoom,reset,box_select,poly_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=400, height=400, title='SDSS g-r vs u-g', webgl=True)
left.x('umg', 'gmr', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=400, height=400, title='SDSS i-z vs r-i')
right.x('rmi', 'imz', source=source)

p = gridplot([[left, right]])

show(p)

See many examples of configuring plot tools at http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html

## Interacting with Glue

In [ ]:
#import glue

In [ ]:
# Quick way to launch Glue
#from glue import qglue
#qglue()

Here we'll interact with Glue from the notebook.

In [ ]:
import astropy.io.fits as fits
hdu = fits.open('data/w5.fits')

In [ ]:
hdu[0].header

In [ ]:
from astropy.table import Table

In [ ]:
catalog = Table.read('data/w5_psc.vot')

In [ ]:
wisecat = Table.read('data/w5_wise.tbl', format='ipac')

In [ ]:
%gui qt

In [ ]:
#qglue(catalog=catalog, image=hdu, wisecat=wisecat)

In [ ]:
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication

#load 2 datasets from files
image = load_data('data/w5.fits')
catalog = load_data('data/w5_psc.vot')

dc = DataCollection([image, catalog])

# link positional information
dc.add_link(LinkSame(image.id['Right Ascension'], catalog.id['RAJ2000']))
dc.add_link(LinkSame(image.id['Declination'], catalog.id['DEJ2000']))

#start Glue
app = GlueApplication(dc)
app.start()

In [ ]:
dc

In [ ]:
dc[0].components

In [ ]:
dc[0].id['Right Ascension']

In [ ]:
catalog = dc[1]

In [ ]:
j_minus_h = catalog['Jmag'] - catalog['Hmag']

In [ ]:
catalog['jmh'] = j_minus_h

In [ ]:
hmag = catalog.id['Hmag']
jmag = catalog.id['Jmag']

In [ ]:
jmhred = (jmag - hmag) > 1.5
dc.new_subset_group('j - h > 1.5', mid_mag)

In [ ]:
dc.subset_groups

In [ ]:
dc.subset_groups[2].label

In [ ]:
catalog.subsets

In [ ]:
catalog.subsets[2]['Jmag']

In [ ]:
catalog.subsets[2].to_mask()

In [ ]:
new_catalog = catalog[catalog.subsets[2].to_mask()]